In [2]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.3/286.3 kB 7.2 MB/s eta 0:00:00


In [16]:
import anthropic

In [17]:
client = anthropic.Anthropic(api_key="вставте ваш ключ для API :) ")

In [18]:
from google.colab import files
uploaded = files.upload()

Saving exit_button_spec.md to exit_button_spec (1).md


In [19]:
doc_path = list(uploaded.keys())[0]
with open(doc_path, "r", encoding="utf-8") as f:
    documentation = f.read()

In [21]:
# Промпт: генерація тест-кейсів на основі документації
prompt = f"""
You are a QA engineer. Based on the following documentation, generate test cases
in a table with the columns: ID | Title | Preconditions | Steps | Expected Result.

Documentation:
{documentation}
"""

In [24]:
client.models.list()

SyncPage[ModelInfo](data=[ModelInfo(id='claude-opus-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4', type='model'), ModelInfo(id='claude-sonnet-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4', type='model'), ModelInfo(id='claude-3-7-sonnet-20250219', created_at=datetime.datetime(2025, 2, 24, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 3.7', type='model'), ModelInfo(id='claude-3-5-sonnet-20241022', created_at=datetime.datetime(2024, 10, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 3.5 (New)', type='model'), ModelInfo(id='claude-3-5-haiku-20241022', created_at=datetime.datetime(2024, 10, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Haiku 3.5', type='model'), ModelInfo(id='claude-3-5-sonnet-20240620', created_at=datetime.datetime(2024, 6, 20, 0, 0, tzinfo=datetime.timezone.utc), dis

In [25]:
# Виклик Claude
response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=2000,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [26]:
# Отриманий результат
raw_text = response.content[0].text
print("🔎 Claude response preview:\n")
print(raw_text[:1000])  # вивід початку для перевірки

🔎 Claude response preview:

Based on the documentation, here are the test cases for the Exit to Main Menu Button feature:

| ID | Title | Preconditions | Steps | Expected Result |
|----|-------|---------------|-------|-----------------|
| TC001 | Verify Exit to Menu button is displayed in pause menu | - Game is running<br>- Player is in a non-combat moment<br>- Pause menu is accessible | 1. Start the game and enter gameplay<br>2. Open the pause menu | - "Exit to Menu" button is visible<br>- Button is positioned below the "Resume" button<br>- Button displays with 16sp font size |
| TC002 | Verify Exit to Menu button tap area | - Game is running<br>- Pause menu is open | 1. Open pause menu<br>2. Attempt to tap the "Exit to Menu" button in various areas around the button | - Button responds to taps within minimum 48x48 dp area<br>- Taps outside the button area do not trigger the button |
| TC003 | Verify confirmation dialog appears when Exit to Menu is pressed | - Game is running<br>- Pau

In [27]:
# Парсинг Markdown-таблиці
import pandas as pd
from io import StringIO

lines = raw_text.strip().split("\n")

# Шукаємо рядки з таблицею
table_lines = [line for line in lines if "|" in line and not line.strip().startswith("---")]

# Парсимо в DataFrame
table_text = "\n".join(table_lines)
df = pd.read_csv(StringIO(table_text), sep="|")
df.columns = [c.strip() for c in df.columns]
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

<ipython-input-27-27337af8bb39>:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [29]:
# Збереження у Excel
output_path = "claude_test_cases.xlsx"
df.to_excel(output_path, index=False)

from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>